# Django Models

## Django Models and the Migrations Workflow

### Create Django Models


7.1. Django model class definition in models.py


In [ ]:
from __future__ import unicode_literals
from django.utils.encoding import python_2_unicode_compatible
from django.db import models


@python_2_unicode_compatible
class Store(models.Model):
    #id = models.AutoField(primary_key=True) # Added by default, not required explicitly
    name = models.CharField(max_length=30)
    address = models.CharField(max_length=30)
    city = models.CharField(max_length=30)
    state = models.CharField(max_length=2)
    #objects = models.Manager()# Added by default, to required explicitly
    
    def __str__(self):
        return "%s (%s,%s)" % (self.name, self.city, self.state)


### Migrations and the Django Model Workflow


7.3. Django makemigrations command to create migration file for changes made to models.py


In [ ]:
"""
!python manage.py makemigrations stores
"""

7.4. Django sqlmigrate command to preview SQL generated by migration file

In [ ]:
"""
!python manage.py sqlmigrate stores 0001
"""

7.5. Django migrate command to execute migration files on database


In [ ]:
"""
!python manage.py migrate stores
"""

### Predetermined Values: default, auto_now, auto_now_add, and choices

7.6. Django model default option use


In [ ]:
def default_city():
    return "San Diego"

class Store(models.Model):
    name = models.CharField(max_length=30)
    address = models.CharField(max_length=30)
    city = models.CharField(max_length=30, default=default_city)
    state = models.CharField(max_length=2, default='CA')


7.7. Django model default options for dates and times, as well as auto_now and auto_now_add use


In [ ]:
from datetime import date
from django.utils import timezone


class Store(models.Model):
    name = models.CharField(max_length=30)
    address = models.CharField(max_length=30)
    date = models.DateField(default=date.today)
    datetime = models.DateTimeField(default=timezone.now)
    date_lastupdated = models.DateField(auto_now=True)
    date_added = models.DateField(auto_now_add=True)
    timestamp_lastupdated = models.DateTimeField(auto_now=True)
    timestamp_added = models.DateTimeField(auto_now_add=True)


7.8. Django model choices option


In [ ]:
ITEM_SIZES = (
    ('S', 'Small'), 
    ('M', 'Medium'), 
    ('L', 'Large'), 
    ('P', 'Portion'), 
)

class Menu(models.Model):
    name = models.CharField(max_length=30)

class Item(models.Model):
    menu = models.ForeignKey(Menu, on_delete=models.CASCADE)
    name = models.CharField(max_length=30)
    description = models.CharField(max_length=100)
    size = models.CharField(choices=ITEM_SIZES, max_length=1)


7.9. Django model help_text option

In [ ]:
ITEM_SIZES = (
    ('S', 'Small'), 
    ('M', 'Medium'), 
    ('L', 'Large'), 
    ('P', 'Portion'), 
)

class Menu(models.Model):
    name = models.CharField(max_length=30)

class Item(models.Model):
    menu = models.ForeignKey(Menu, on_delete=models.CASCADE)
    name = models.CharField(max_length=30)
    description = models.CharField(max_length=100, help_text="Ensure you provide some description of the ingredients")
    size = models.CharField(choices=ITEM_SIZES, max_length=1)
    calories = models.IntegerField(help_text="Calorie count should reflect <b>size</b> of the item")


### Database Definition Language (DDL) Values: db_column, db_index, db_tablespace, primary_key

7.10. Django model field validators option with built-in and custom validator

In [ ]:
from django.core.validators import MinLengthValidator
from django.core.exceptions import ValidationError

ITEM_SIZES = (
    ('S', 'Small'), 
    ('M', 'Medium'), 
    ('L', 'Large'), 
    ('P', 'Portion'), 
)

def calorie_watcher(value):
    if value > 5000:
        raise ValidationError(
            ('Whoa! calories are %(value)s ? We try to serve healthy food, try something less than 5000!'), 
            params={'value': value}, 
        )
    if value < 0:
        raise ValidationError(
            ('Strange calories are %(value)s ? This can\'t be, value must be greater than 0'), 
            params={'value': value}, 
        )

class Menu(models.Model):
    name = models.CharField(max_length=30)

class Item(models.Model):
    menu = models.ForeignKey(Menu, on_delete=models.CASCADE)
    name = models.CharField(max_length=30, validators=[MinLengthValidator(5)])
    description = models.CharField(max_length=100)
    size = models.CharField(choices=ITEM_SIZES, max_length=1)
    calories = models.IntegerField(validators=[calorie_watcher])


## Django Model Default and Custom Behaviors

7.11. Django model use of the save() method

In [ ]:
from coffeehouse.stores.models import Store

store_corporate = Store(
    name='Corporate', 
    address='624 Broadway', 
    city='San Diego', 
    state='CA', 
    email='corporate@coffeehouse.com'
    )
store_corporate.save()
store_corporate.city = '625 Broadway'
store_corporate.save()


7.12. Django model with custom save() method

In [ ]:
class Store(models.Model):
    name = models.CharField(max_length=30)
    address = models.CharField(max_length=30)
    city = models.CharField(max_length=30)
    state = models.CharField(max_length=2)

    def save(self, *args, **kwargs):
        # Do custom logic here (e.g. validation, logging, call third party service)
        # Run default save() method
        super(Store, self).save(*args, **kwargs)


7.13. Django model use of validation clean_fields() method


In [ ]:
class Store(models.Model):
    name = models.CharField(max_length=30)
    address = models.CharField(max_length=30, unique=True)
    city = models.CharField(max_length=30)
    state = models.CharField(max_length=2)

# Create a model Store instance, that violates the max_length rule
store_corporate = Store(
    name='This is a very long name for the Corporate store that exceeds the 30 character limit', 
    address='624 Broadway', 
    city='San Diego', 
    state='AZ', 
    email='corporate@coffeehouse.com'
)

# No error yet. You could call save() and let the database reject the instance
# But you can also validate at the Django/Python level with the clean_fields() method
store_corporate.clean_fields()


7-14. Django model use of validation clean() method

In [ ]:
class Store(models.Model):
    name = models.CharField(max_length=30)
    address = models.CharField(max_length=30, unique=True)
    city = models.CharField(max_length=30)
    state = models.CharField(max_length=2)

    def clean(self):
        if self.city == 'San Diego' and self.state != 'CA':
            raise ValidationError(
                'Wait San Diego is CA!, are you sure there is another San Diego in %s ?' % self.state
            )

# Create a model Store instance, that violates city/state rule
store_corporate = Store(name='Corporate', address='624 Broadway', city='San Diego', state='AZ', 
email='corporate@coffeehouse.com')
# To enforce more complex rules call the clean() method implemented on a model
store_corporate.clean()


7.15. Django model use of validation clean_unique() method with unique* fields

In [ ]:
class Store(models.Model):
    name = models.CharField(max_length=30)
    address = models.CharField(max_length=30, unique=True)
    city = models.CharField(max_length=30)
    state = models.CharField(max_length=2)

# Create a model Store instance
store_corporate = Store(
    name='Downtown', 
    address='624 Broadway', 
    city='San Diego', 
    state='AZ', 
    email='corporate@coffeehouse.com'
)
store_corporate.save()

# Create another instance to violate uniqueness of address field
store_uptown = Store(
    name='Uptown', 
    address='624 Broadway', 
    city='San Diego', 
    state='CA'
)
# You could call save() and let the database reject the instance.
# But you can also validate at the Django/Python level with the validate_unique() method
store_uptown.validate_unique()


7.16. Django model use of validation clean_unique() method with Meta unique_together


In [ ]:
class Store(models.Model):
    name = models.CharField(max_length=30)
    address = models.CharField(max_length=30, unique=True)
    city = models.CharField(max_length=30)
    state = models.CharField(max_length=2)
    email = models.EmailField()

    class Meta:
        unique_together = ("name", "email")

# Create instance to show use of validate_unique() via Meta option
store_downtown_horton = Store(
    name='Downtown', 
    address='Horton Plaza', 
    city='San Diego', 
    state='CA', 
    email='downtown@coffeehouse.com'
)
store_downtown_horton.save()

# Create additional instance that violated unique_together rule in Meta class
store_downtown_fv = Store(
    name='Downtown', 
    address='Fashion Valley', 
    city='San Diego', 
    state='CA', 
    email='downtown@coffeehouse.com'
)

# You could call save() and let the database reject the instance but lets use validate_unique
store_downtown_fv.validate_unique()



7.17. Django model with custom method


In [ ]:
def geocoding_method():
    return 
    
class Store(models.Model):
    name = models.CharField(max_length=30)
    address = models.CharField(max_length=30)
    city = models.CharField(max_length=30)
    state = models.CharField(max_length=2)

    def latitude_longitude(self):
        # Call remote service to get latitude & longitude
        latitude, longitude = geocoding_method(self.address, self.city, self.state)
        return latitude, longitude


7.18. Django default model manager renamed

In [ ]:
class Store(models.Model):
    name = models.CharField(max_length=30)
    address = models.CharField(max_length=30)
    city = models.CharField(max_length=30)
    state = models.CharField(max_length=2)
    mgr = models.Manager()


7.19. Django model with Meta class and ordering option


In [ ]:
class Store(models.Model):
    name = models.CharField(max_length=30)
    address = models.CharField(max_length=30)
    city = models.CharField(max_length=30)
    state = models.CharField(max_length=2)

    class Meta:
        ordering = ['-state']


7.20. Django model with meta class and index option


In [ ]:
class Store(models.Model):
    name = models.CharField(max_length=30)
    address = models.CharField(max_length=30)
    city = models.CharField(max_length=30)
    state = models.CharField(max_length=2)

    class Meta:
        indexes = [
            models.Index(fields=['city','state']),
            models.Index(fields=['city'], name='city_idx')
        ]


7.21. Django model abstract option


In [ ]:
class Item(models.Model):
    name = models.CharField(max_length=30)
    description = models.CharField(max_length=100)

    class Meta:
        abstract = True

class Drink(Item):
    mililiters = models.IntegerField()

## Relationships in Django Models


7.22. One to many Django model relationship

In [ ]:
class Menu(models.Model):
    name = models.CharField(max_length=30)

class Item(models.Model):
    menu = models.ForeignKey(Menu)
    name = models.CharField(max_length=30)
    description = models.CharField(max_length=100)


7.23. Many to many Django model relationship

In [ ]:
class Amenity(models.Model):
    name = models.CharField(max_length=30)
    description = models.CharField(max_length=100)

class Store(models.Model):
    name = models.CharField(max_length=30)
    address = models.CharField(max_length=30)
    city = models.CharField(max_length=30)
    state = models.CharField(max_length=2)
    email = models.EmailField()
    amenities = models.ManyToManyField(Amenity, blank=True)


7.24. One to one Django model relationship


In [ ]:
class Menu(models.Model):
    name = models.CharField(max_length=30)

class Item(models.Model):
    menu = models.ForeignKey(Menu)
    name = models.CharField(max_length=30)
    description = models.CharField(max_length=100)
    calories = models.IntegerField()
    price = models.FloatField()

class Drink(models.Model):
    item = models.OneToOneField(Item, on_delete=models.CASCADE, primary_key=True)
    caffeine = models.IntegerField()


7.25. One to many Django model relationship with self-referencing model

In [ ]:
class Category(models.Model):
    menu = models.ForeignKey('self')

class Person(models.Model):
    relatives = models.ManyToManyField('self')


7.26. One to many Django model relationship with reverse relationship references


In [ ]:
class Menu(models.Model):
    name = models.CharField(max_length=30)

class Item(models.Model):
    menu = models.ForeignKey(Menu, on_delete=models.CASCADE)
    name = models.CharField(max_length=30)
    description = models.CharField(max_length=100)
    price = models.FloatField(blank=True, null=True)

breakfast = Menu.objects.get(name='Breakfast')

# Direct access
all_items_with_breakfast_menu = Item.objects.filter(menu=breakfast)

# Reverse access through instance
same_all_items_with_breakfast_menu = breakfast.item_set.all()


7.27. One to many Django model relationship with reverse relationship queries


In [ ]:
# Based on models from listing 7.26

# Direct access, Item records with price higher than 1
Item.objects.filter(price__gt=1)

# Reverse access query, Menu records with Item price higher than 1
Menu.objects.filter(item__price__gt=1)


## Django Model Transactions


In [ ]:
from django.db import transaction

7.28. Selectively activate and deactivate atomic requests with @non_atomic_requests and @atomic

In [ ]:

# When ATOMIC_REQUESTS=True you can individually disable atomic requests

def data_operation_1(): return
def data_operation_2(): return
def data_operation_3(): return

@transaction.non_atomic_requests
def index(request):
    # Data operations with transactions commit/rollback individually
    # Failure of one operation does not influence other
    data_operation_1()
    data_operation_2()
    data_operation_3()

# When ATOMIC_REQUESTS=False you can individually enable atomic requests

@transaction.atomic
def detail(request):
    # Start transaction.
    # Failure of any operation, rollbacks other operations
    data_operation_1()
    data_operation_2()
    data_operation_3()
    # Commit transaction if all operation successful

7.29. Transactions with context managers


In [ ]:
def data_operation_standalone(): return

def data_operation_standalone2(): return

def login(request):
    # With AUTO_COMMIT=True and ATOMIC_REQUEST=False
    # Data operation runs in its own transaction due to AUTO_COMMIT=True
    data_operation_standalone()
    # Open new transaction with context manager
    with transaction.atomic():
        # Start transaction.
        # Failure of any operation, rollbacks other operations
        data_operation_1()
        data_operation_2()
        data_operation_3()
        # Commit transaction if all operation successful

# Data operation runs in its own transaction due to AUTO_COMMIT=True
data_operation_standalone2()

## Django Model Migrations


In [ ]:
from django.db import migrations, models

7.30. Django migration file basic structure

In [ ]:
class Migration(migrations.Migration):
    initial = True
    replaces = []
    dependencies = []
    operations = []


## Django Model Initial Data Setup


7.31. Create empty Django migration file to load initial data for Django model

In [ ]:
!python manage.py makemigrations --empty stores


Listing 7-32. Load initial data with hard-coded data in Django migration file


In [ ]:
# -*- coding: utf-8 -*-

def load_stores(apps, schema_editor):
    Store = apps.get_model("stores", "Store")
    store_corporate = Store(
        id=0, 
        name='Corporate', 
        address='624 Broadway', 
        city='San Diego', 
        state='CA', 
        email='corporate@coffeehouse.com'
    )
    store_corporate.save()
    store_downtown = Store(
        id=1, 
        name='Downtown', 
        address='Horton Plaza', 
        city='San Diego', 
        state='CA', 
        email='downtown@coffeehouse.com'
    )
    store_downtown.save()
    store_uptown = Store(
        id=2, 
        name='Uptown', 
        address='1240 University Ave', 
        city='San Diego', 
        state='CA', 
        email='uptown@coffeehouse.com'
    )
    store_uptown.save()
    store_midtown = Store(
        id=3, 
        name='Midtown', 
        address='784 W Washington St', 
        city='San Diego', 
        state='CA', 
        email='midtown@coffeehouse.com'
    )
    store_midtown.save()


def delete_stores(apps, schema_editor):
    Store = apps.get_model("stores", "Store")
    Store.objects.all().delete()


class Migration(migrations.Migration):
    dependencies = [('stores', '0001_initial'),]
    operations = [migrations.RunPython(load_stores, delete_stores), ]


7.33. SQL script with SQL statements


In [ ]:
INSERT INTO stores_store (id, name, address, city, state, email)  
VALUES (0, 'Corporate', '624 Broadway', 'San Diego', 'CA', 'corporate@coffeehouse.com');

INSERT INTO stores_store (id, name, address, city, state, email) 
VALUES (1, 'Downtown', 'Horton Plaza', 'San Diego', 'CA', 'downtown@coffeehouse.com');

INSERT INTO stores_store (id, name, address, city, state, email) 
VALUES (2, 'Uptown', '1240  University Ave', 'San Diego', 'CA', 'uptown@coffeehouse.com');

INSERT INTO stores_store (id, name, address, city, state, email) 
VALUES (3, 'Midtown', '784 WWashington St', 'San Diego', 'CA', 'midtown@coffeehouse.com');


7.34. Load initial data with SQL script in Django migration file

In [ ]:
# -*- coding: utf-8 -*-

def load_stores_from_sql():
    from coffeehouse.settings import PROJECT_DIR
    import os
    sql_statements = open(os.path.join(PROJECT_DIR, 'stores/sql/store.sql'), 'r').read()
    return sql_statements

def delete_stores_with_sql():
    return 'DELETE from stores_store;'

class Migration(migrations.Migration):
    dependencies = [('stores', '0001_initial'), ]
    operations = [migrations.RunSQL(load_stores_from_sql(), delete_stores_with_sql()), ]


7.35. Django fixture file with JSON structure


In [ ]:
[
    {
        "fields": {
            "city": "San Diego",
            "state": "CA",
            "email": "corporate@coffeehouse.com",
            "name": "Corporate",
            "address": "624 Broadway"
        },
        "model": "stores.store",
        "pk": 0
    },
    {
        "fields": {
            "city": "San Diego",
            "state": "CA",
            "email": "downtown@coffeehouse.com",
            "name": "Downtown",
            "address": "Horton Plaza"
        },
        "model": "stores.store",
        "pk": 1
    }
]

7.36. Load initial data from Django fixture file in Django migration file

In [ ]:
from django.core.management import call_command

def load_stores_from_fixture(apps, schema_editor):
    call_command("loaddata", "store")

def delete_stores(apps, schema_editor):
    Store = apps.get_model("stores", "Store")
    Store.objects.all().delete()

class Migration(migrations.Migration):
    dependencies = [('stores', '0001_initial'), ]
    operations = [migrations.RunPython(load_stores_from_fixture, delete_stores), ]


## Django Model Signals


In [ ]:
import logging
from django.dispatch import receiver, Signal
from django.db.models.signals import pre_save
from django.apps import AppConfig


7.37. Basic syntax to listen for Django signals


In [ ]:
@receiver('<signal_to_listen_for_from_django_core_signals>', sender='<model_class_to_listen_to>')
def method_with_logic_to_run_when_signal_is_emitted(sender, **kwargs):
    # Logic when signal is emitted
    # Access sender & kwargs to get info on model that emitted signal
    pass


7.38. Listen for Django pre_save signal on Item model in signals.py


In [ ]:
stdlogger = logging.getLogger(__name__)


@receiver(pre_save, sender='items.Item')
def run_before_saving(sender, **kwargs):
    stdlogger.info("Start pre_save Item in signals.py under items app")
    stdlogger.info("sender %s" % (sender))
    stdlogger.info("kwargs %s" % str(kwargs))


7.39. Django apps.py with custom ready() method to load signals.py


In [ ]:
class ItemsConfig(AppConfig):
    name = 'coffeehouse.items'
    def ready(self):
        import coffeehouse.items.signals


Listing 7-40. Django configuration options to load apps.py


In [ ]:
# Option 1) Declare apps.py class as part of INSTALLED_APPS

# In settings.py
INSTALLED_APPS = [
    'coffeehouse.items.apps.ItemsConfig',
]

# Option 2) Declare default_app_config inside the __init__ file of the app

# In /coffeehouse/items/__init__.py
default_app_config = 'coffeehouse.items.apps.ItemsConfig'

7.41. Django model emitting custom signal


In [ ]:
# In signals.py

order_complete = Signal(providng_args=["customer","barista"])
store_closed = Signal(providing_args=["employee"])

# In models.py

from coffeehouse.stores.signals import store_closed

class Store(models.Model):
    name = models.CharField(max_length=30)
    address = models.CharField(max_length=30, unique=True)

    def closing(self, employee):
        store_closed.send(sender=self.__class__, employee=employee)

# In some other file that uses the class Store

@receiver(store_closed)
def run_when_store_is_closed(sender,**kwargs):
    stdlogger.info("Start store_closed Store in signals.py under stores app")
    stdlogger.info("sender %s" % (sender))
    stdlogger.info("kwargs %s" % str(kwargs))


## Django Models Outside of models.py


7.42. Django apps with models stored under models directory


```
+---+
    |
    +-stores(app)-+
                  +-__init__.py
                  +-models.py
                  +-tests.py
                  +-views.py
                  +-apps.py
                  +-models-+
                           |
                           +-__init__.py
                           +-menus.py
                           +-equipment.py
                           +-personnel.py
```


7.43. Django apps with models stored under custom directories

```
+---+
    |
    +-stores(app)-+
                  +-__init__.py
                  +-models.py
                  +-tests.py
                  +-views.py
                  +-apps.py
                  +-menus+
                  |      +-__init__.py
                  |      +-breakfast.py
                  |
                  +-equipment+
                             +-__init__.py
                             +-kitchen.py
```


## Django Models and Multiple Databases


7.44. Django multiple DATABASES definitions in settings.py


In [ ]:
DATABASES = {
    'default': {},
    'devops': {},
    'analytics': {},
    'warehouse': {},
}

7.45. Django database router to store core app models in devops database and all other models in default database


In [ ]:
class DatabaseForDevOps(object):
    def db_for_read(self, model, **hints):
        return (
            'devops' 
            if model._meta.app_label in ['auth','admin','sessions','contenttypes'] else 
            None
        )

    def db_for_write(self, model, **hints):
        return (
            'devops' 
            if model._meta.app_label in ['auth','admin','sessions','contenttypes'] else 
            None
        )

    def allow_relation(self, obj1, obj2, **hints):
        condition1 = obj1._meta.app_label in ['auth','admin','sessions','contenttypes']
        condition2 = obj2._meta.app_label in ['auth','admin','sessions','contenttypes']
        if condition1 and condition2:
            return True
        elif  not condition1 or not condition2:
            return None
        return None

    def allow_migrate(self, db, app_label, model_name=None, **hints):
        if db == 'devops':
            return (
                True
                if app_label in ['auth','admin','sessions','contenttypes'] else
                False
            )
        elif app_label in ['auth','admin','sessions','contenttypes']:
            return False
        return None
